**Skills & Benefits Breakdown**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import libraries
import pandas as pd

In [91]:
# Load datasets from Google Drive
skills = pd.read_csv('/content/drive/My Drive/Project data set/skills.csv')
job_skills = pd.read_csv('/content/drive/My Drive/Project data set/job_skills.csv')
benefits = pd.read_csv('/content/drive/My Drive/Project data set/benefits.csv')
salaries = pd.read_csv('/content/drive/My Drive/Project data set/salaries.csv')
postings = pd.read_csv('/content/drive/My Drive/Project data set/postings.csv')
job_industries = pd.read_csv('/content/drive/My Drive/Project data set/job_industries.csv')
industries = pd.read_csv('/content/drive/My Drive/Project data set/industries.csv')

In [5]:
skills.head(2)

,skill_abr,skill_name
0,ART,Art/Creative
1,DSGN,Design


In [6]:
job_skills.head(2)

,job_id,skill_abr
0,3884428798,MRKT
1,3884428798,PR


In [92]:
benefits.head(2)

,job_id,inferred,type
0,3887473071,0,Medical insurance
1,3887473071,0,Vision insurance


In [95]:
benefits = benefits.rename(columns={'type': 'benefit'}).replace({
    '401(k)': 'Retirement Plan',
    'Pension plan': 'Retirement Plan'
})

benefits.head(2)

,job_id,inferred,benefit
0,3887473071,0,Medical insurance
1,3887473071,0,Vision insurance


In [37]:
salaries.head(2)

,salary_id,job_id,max_salary,med_salary,min_salary,pay_period,currency,compensation_type
0,1,3884428798,NaN,20.0,NaN,HOURLY,USD,BASE_SALARY
1,2,3887470552,25.0,NaN,23.0,HOURLY,USD,BASE_SALARY


In [38]:
# Calculate average salary
salaries['avg_salary'] = salaries[['min_salary', 'med_salary', 'max_salary']].mean(axis=1)

In [44]:
postings.head(2)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0


In [96]:
#merge data set
df = postings.merge(salaries, on= 'job_id', how= 'left')\
             .merge(benefits, on ='job_id', how='left')\
             .merge(job_skills, on='job_id', how='left')\
             .merge(skills, on='skill_abr', how='left')

In [97]:
print(df.head())

     job_id            company_name                              title  \
0    921716   Corcoran Sawyer Smith              Marketing Coordinator   
1    921716   Corcoran Sawyer Smith              Marketing Coordinator   
2   1829192                     NaN  Mental Health Therapist/Counselor   
3  10998357  The National Exemplar         Assitant Restaurant Manager   
4  10998357  The National Exemplar         Assitant Restaurant Manager   

                                         description  max_salary_x  \
0  Job descriptionA leading real estate firm in N...          20.0   
1  Job descriptionA leading real estate firm in N...          20.0   
2  At Aspen Therapy and Wellness , we are committ...          50.0   
3  The National Exemplar is accepting application...       65000.0   
4  The National Exemplar is accepting application...       65000.0   

  pay_period_x          location  company_id  views  med_salary_x  ...  \
0       HOURLY     Princeton, NJ   2774458.0   20.0         

In [98]:
df.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary_x',
       'pay_period_x', 'location', 'company_id', 'views', 'med_salary_x',
       'min_salary_x', 'formatted_work_type', 'applies',
       'original_listed_time', 'remote_allowed', 'job_posting_url',
       'application_url', 'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency_x',
       'compensation_type_x', 'normalized_salary', 'zip_code', 'fips',
       'salary_id', 'max_salary_y', 'med_salary_y', 'min_salary_y',
       'pay_period_y', 'currency_y', 'compensation_type_y', 'inferred',
       'benefit', 'skill_abr', 'skill_name'],
      dtype='object')

**top most common skills**

In [99]:
#Top 20 Most Common Skills
top_skills = df['skill_name'].value_counts().head(10).reset_index()
print("Top 20 Most Common Skills:\n", top_skills)

Top 20 Most Common Skills:
                skill_name  count
0  Information Technology  31121
1                   Sales  27801
2              Management  25588
3           Manufacturing  21842
4    Health Care Provider  20369
5    Business Development  18060
6             Engineering  16923
7                   Other  15107
8                 Finance  11689
9     Accounting/Auditing   7841


**TOP SKILLS IN HIGH PAYING ROLES**

In [100]:
#Top 20 Highest Paying Skills (with minimum at leats 50 times)
skill_salary_summary = df.groupby('skill_name').agg(
    count=('job_id', 'count'),
    avg_salary=('normalized_salary', 'mean')
).query('count >= 50').sort_values('avg_salary', ascending=False).head(10).reset_index()
print("\ntop skills in high-paying roles:\n\n", skill_salary_summary)


top skills in high-paying roles:

                skill_name  count     avg_salary
0                   Legal   3632  577545.002788
1  Information Technology  31121  474477.329617
2                Training   2695  344314.813461
3             Engineering  16923  332791.297251
4               Education   2780  326906.162694
5                   Other  15107  309397.686314
6    Health Care Provider  20369  225544.999356
7     Accounting/Auditing   7841  192843.715175
8                 Finance  11689  180239.523730
9               Marketing   7059  141210.768382


**HOW SKILLS VARIES BY INDUSTRY OR LOCATION**

In [102]:
# View the job industry data
job_industries.head(2)

,job_id,industry_id
0,3884428798,82
1,3887473071,48


In [82]:
# View the industry data
industries.head(2)

,industry_id,industry_name
0,1,Defense and Space Manufacturing
1,3,Computer Hardware Manufacturing


In [84]:
# merge industries and job industries
industry = industries.merge(job_industries, on='industry_id', how='left')
industry.head(1)

,industry_id,industry_name,job_id
0,1,Defense and Space Manufacturing,3887470617


In [105]:
#merge with the main data set
jobs = df.merge(industry, on='job_id', how='left')
jobs.head(2)

,job_id,company_name,title,description,max_salary_x,pay_period_x,location,company_id,views,med_salary_x,...,pay_period_y,currency_y,compensation_type_y,inferred,benefit,skill_abr,skill_name,remote_status,industry_id,industry_name
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,HOURLY,USD,BASE_SALARY,NaN,NaN,MRKT,Marketing,NaN,44.0,Real Estate
1,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,HOURLY,USD,BASE_SALARY,NaN,NaN,SALE,Sales,NaN,44.0,Real Estate


In [86]:
#Skills by industries
skills_by_industry = jobs.groupby(['industry_name', 'skill_name']).size().reset_index(name='count')

# Show top 10 combinations
skills_by_industry.sort_values('count', ascending=False).head(10)

,industry_name,skill_name,count
2817,Hospitals and Health Care,Health Care Provider,13893
2961,IT Services and IT Consulting,Information Technology,6833
5457,Retail,Sales,5436
5433,Retail,Business Development,3867
2166,Financial Services,Sales,3728
2149,Financial Services,Finance,3687
6154,Staffing and Recruiting,Health Care Provider,3009
5446,Retail,Management,2751
5996,Software Development,Information Technology,2647
2956,IT Services and IT Consulting,Engineering,2634


In [89]:
#Skills by location
skills_by_location = jobs.groupby(['location', 'skill_name']).size().reset_index(name='count')

# Show top 10 combinations
skills_by_location.sort_values('count', ascending=False).head(10)

,location,skill_name,count
44669,United States,Information Technology,4590
44682,United States,Sales,3134
44658,United States,Business Development,2352
44664,United States,Engineering,2137
44665,United States,Finance,1372
44673,United States,Marketing,1245
44659,United States,Consulting,1107
30969,"New York, NY",Sales,1101
44671,United States,Management,936
44674,United States,Other,929


**BENEFITS WITH REMOTE JOBS**

In [103]:
benefit_counts = benefits['benefit'].value_counts().head(15).reset_index()
print("\nTop 15 Reported Benefits:\n", benefit_counts)


Top 15 Reported Benefits:
                     benefit  count
0           Retirement Plan  25137
1         Medical insurance   9873
2          Vision insurance   9309
3      Disability insurance   7930
4          Dental insurance   6868
5        Tuition assistance   2614
6         Commuter benefits   2226
7      Paid maternity leave   1808
8      Paid paternity leave   1540
9   Student loan assistance    365
10       Child care support    273


In [104]:
# Add a remote_status column for clarity
df['remote_status'] = df['remote_allowed'].map({1: 'Remote', 0: 'Not Remote'})

# Group by benefit and remote_status
benefit_remote = df.groupby(['benefit', 'remote_status']).size().unstack(fill_value=0)

# Add total counts to sort
benefit_remote['total'] = benefit_remote.sum(axis=1)

# Get top 10 benefits by total frequency
top_benefits = benefit_remote.sort_values('total', ascending=False).head(10)

# Drop the 'total' column for final clean view
top_benefits = top_benefits.drop(columns='total')

print("\nTop 10 Benefits by Remote vs Not Remote Jobs:\n")
print(top_benefits)


Top 10 Benefits by Remote vs Not Remote Jobs:

remote_status         Remote
benefit                     
Retirement Plan         4173
Medical insurance       2419
Vision insurance        2283
Dental insurance        1871
Disability insurance    1304
Paid maternity leave     696
Paid paternity leave     632
Commuter benefits        430
Tuition assistance       363
Child care support        93
